In [1]:
%run Featurize_Input.ipynb

Using backend: pytorch


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import NNConv
from rdkit import Chem
from tqdm.notebook import tqdm
from dgl.dataloading import GraphDataLoader as graphdataloader

In [7]:
from dgllife.model.model_zoo.mpnn_predictor import MPNNPredictor

In [3]:
class Model(nn.Module):
    def __init__(self, hidden_feat_size, out_feat_size, activ):
        super(Model, self).__init__()
        self.hidden_layer = hidden_feat_size 
        self.layer1 = NNConv(
            in_feats = hidden_feat_size,
            out_feats = out_feat_size,
            edge_func = activ,
            aggregator_type = 'sum'
        )
        '''self.layer2 = torch.nn.Sequential(
            nn.Linear(hidden_feat_size, hidden_feat_size),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(hidden_feat_size, out_feat_size)
        )'''
        
    def forward(self, x, nodes, edges):
        #g = nn.Linear(len(x), self.hidden_layer)
        nnconv = self.layer1(x, nodes, edges)
        #seq = self.layer2(g)
        return nnconv

In [4]:
#lin = nn.Linear(100, 100)
#def edge_func(efeat):
#    return lin(efeat)
#model = Model(100, 20, edge_func)

In [8]:
model = MPNNPredictor(1,1)

In [11]:
def train(epochs):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    # dataloader
    for epoch in tqdm(range(epochs)):
        # TRAIN 
        model.train()
        running_loss = 0.
        
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            atoms = batch_x.ndata['atomic']
            edges = batch_x.edata['type']
            y_pred = model(batch_x, atoms, edges)
            mse = ((y_pred.reshape(1,-1) - batch_y)**2).sum()
            running_loss += mse.item()
            mse.backward()
            optimizer.step()
            
        running_loss /= len(train_dataset)
            
        # EVAL
        model.eval()
        val_score = 0.

#         for batch_x_val, batch_y_val in val_dataloader:
#             optimizer.zero_grad()
#             atoms = batch_x.ndata['atomic']
#             edges = batch_x.edata['type']
#             y_pred = model(batch_x, atoms, edges)
#             mse = ((y_pred.reshape(-1) - batch_y_val)**2).sum()
#             val_score += mse.item()
#             mse.backward()
#             optimizer.step()
#             val_score /= len(val_dataset)

#         if not best_score:
#             best_score = val_score
#             torch.save(model, 'best-model.pt') 
#         if val_score < best_score:
#             best_score = val_score
#             torch.save(model, 'best-model.pt')
    print("Train loss: ", running_loss, "Validation loss: ", val_score, "Best Validation loss: ", best_score)

In [ ]:
train(100)

  0%|          | 0/100 [00:00<?, ?it/s]